gemaakt voor LIWO door [David Haasnoot](d.haasnoot@hkv.nl)

In [ ]:
import dotenv
from pathlib import Path

In [ ]:
from update_local_bulk_LDO import haal_scenarios_op, haal_token_op

Zie `readme.md` voor uitgebreid stappen plan voor het aanmaken van een api key.

In het kort
- Login & ga naar https://ldo.overstromingsinformatie.nl/auth/
- Onder `V1`, `POST auth/v1/personalapikeys`, klik op `try it out` en vervang body met:
```json
{
  "scope": "*:readwrite",
  "name": "personalAPIkeyLIWOexport...",
  "expiry_date": "2029-12-31T23:59:59.037Z",
  "revoked": false
}
```
- Pas de datum en name aan waar nodig, klik op `Excecute` in het blauw. In de response body staat: 
```json
{
...
  "key": "xxxxxx.yyyyyyyyyyyyyyy",
...
}
```
- Bewaar die hele `'key'` in een bestand die `.env` heet, zie `.env.example` voor het formaat.
- Afhankelijk via welke organisatie je toegang hebt, kan het nodig zijn om in de code de `TENANT` variabele aan te passen. Deze kan ook in de `.env` file worden gezet.
  - `TENANT = 0` voor beheerders? 
  - `TENANT = 1` voor LIWO
  - `TENANT = 2` voor RWS
  ...

In [ ]:
# haal de API key op uit de .env file
if dotenv.load_dotenv():
    environmental_variables = dotenv.dotenv_values()
    LDO_api_key = environmental_variables["LDO_api_key"]\
# Of zet hier de API key handmatig in
# LDO_api_key = "abcd"
TENANT : int  = 1 # 0, 1, 2 ...
headers = haal_token_op(LDO_api_key, tenant=TENANT)

In [ ]:
maximum = 10_000
beschikbare_scenario_ids = haal_scenarios_op(maximum, headers)

In [ ]:
len(beschikbare_scenario_ids)

In [ ]:
from update_local_bulk_LDO import vergelijke_nieuwe_en_huidige, export_uit_LDO, voeg_zips_samen_verwijder_ouder

Zet paden goed

In [ ]:
current_dir = Path.cwd()
current_archive = current_dir / "merged_nov_2024.zip"
new_archive = current_dir / "merged_feb_2025_2.zip"

Vergelijkt een huidige export en voegt alleen nieuwe toe

In [ ]:
verwijderde_scenarios, nieuwe_scenarios, df_current_local_LDO = (
    vergelijke_nieuwe_en_huidige(current_archive, beschikbare_scenario_ids)
)

Exporteert de gewenste bestanden

In [ ]:
lst_zips_nieuwe_export = export_uit_LDO(
                                        nieuwe_scenarios=nieuwe_scenarios,
                                        headers=headers,
)

Combineert de bestanden in de zip

In [ ]:
voeg_zips_samen_verwijder_ouder(
                                lst_zips_nieuwe_export,
                                verwijderde_scenarios,
                                df_current_local_LDO,
                                current_archive,
                                new_archive,
)